<h3>ФБ-31 Шваюк Олександра. Лабораторна робота №2</h3>

##### Ініціалізація та перевірка середовища

In [5]:
import datetime
import os
import urllib
import pandas as pd
import glob


print("Setup ok")

Setup ok


#### Завантаження тестових структурованих файлів, що містять значення VHI-індексу.

In [69]:
import os
import urllib.request
import datetime

def load_data(province_code):
    # Встановлення базового шляху для збереження файлів
    base_folder = 'vhi_data'
    if not os.path.isdir(base_folder):
        os.makedirs(base_folder)

    # Формуємо ім'я файлу для збереження даних
    current_date = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    file_name = f'vhi_id__{province_code}__{current_date}.csv'
    file_path = os.path.join(base_folder, file_name)

    # URL для отримання даних
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_code}&year1=1981&year2=2025&type=Mean"

    # Перевірка, чи вже завантажені дані для цієї області
    files_in_folder = [f for f in os.listdir(base_folder) if f.startswith(f'vhi_id__{province_code}__')]
    if files_in_folder:
        print(f"Дані для області {province_code} вже завантажені. Пропускаємо завантаження.")
        return

    # Завантаження даних за допомогою URL
    try:
        response = urllib.request.urlopen(url)
        data = response.read()

        # Збереження завантажених даних у файл
        with open(file_path, 'wb') as file:
            file.write(data)
        print(f"Дані VHI успішно завантажено у файл: {file_path}")

    except Exception as e:
        print(f"Не вдалося завантажити дані для області {province_code}: {e}")

In [70]:
for i in range(1,28):
   load_data(i)

Дані VHI успішно завантажено у файл: vhi_data\vhi_id__1__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__2__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__3__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__4__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__5__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__6__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__7__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__8__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__9__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__10__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__11__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__12__2025-02-18_18-33.csv
Дані VHI успішно завантажено у файл: vhi_data\vhi_id__13__202

#### Встановлення відповідності регіонів за номерами з таблиці

In [132]:
regions_true_id = {
    1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
    6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
    11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
    16: 'Рівненська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
    21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
}

In [133]:
for region_id, region_name in regions_true_id.items():
    print(f"Область {region_id}: {region_name}")

Область 1: Вінницька
Область 2: Волинська
Область 3: Дніпропетровська
Область 4: Донецька
Область 5: Житомирська
Область 6: Закарпатська
Область 7: Запорізька
Область 8: Івано-Франківська
Область 9: Київська
Область 10: Кіровоградська
Область 11: Луганська
Область 12: Львівська
Область 13: Миколаївська
Область 14: Одеська
Область 15: Полтавська
Область 16: Рівненська
Область 17: Сумська
Область 18: Тернопільська
Область 19: Харківська
Область 20: Херсонська
Область 21: Хмельницька
Область 22: Черкаська
Область 23: Чернівецька
Область 24: Чернігівська
Область 25: Республіка Крим


#### Зчитування завантажених текстових файлів у фрейм

In [103]:
def data_frame(folder_path):
    def process_csv(file_path):
        region_num = int(file_path.split('__')[1])
        data = pd.read_csv(file_path, header=1, names=columns)
        data.at[0, 'Year'] = data.at[0, 'Year'][9:]
        data = data.drop(data.index[-1])
        data = data[data['VHI'] != -1]
        data = data.drop(columns=['empty'])
        data.insert(0, 'region_num', region_num, True)
        data['Week'] = data['Week'].astype(int)
        return data

    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    csv_files = glob.glob(f"{folder_path}/*.csv")
    data_frames = [process_csv(file) for file in csv_files]

    combined_data = pd.concat(data_frames).drop_duplicates().reset_index(drop=True)
    combined_data = combined_data[~combined_data.region_num.isin([12, 20])]
    
    region_translation = {
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
        11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17,
        22: 18, 23: 6, 24: 1, 25: 2, 26: 6, 27: 5
    }
    combined_data['region_num'] = combined_data['region_num'].replace(region_translation)

    return combined_data


In [104]:
resultdf = data_frame('vhi_data')

print(resultdf.head(20))

    region_num  Year  Week    SMN     SMT    VCI    TCI    VHI
0           21  1982     1  0.059  258.24  51.11  48.78  49.95
1           21  1982     2  0.063  261.53  55.89  38.20  47.04
2           21  1982     3  0.063  263.45  57.30  32.69  44.99
3           21  1982     4  0.061  265.10  53.96  28.62  41.29
4           21  1982     5  0.058  266.42  46.87  28.57  37.72
5           21  1982     6  0.056  267.47  39.55  30.27  34.91
6           21  1982     7  0.055  268.58  35.19  31.10  33.14
7           21  1982     8  0.057  270.15  33.35  32.09  32.72
8           21  1982     9  0.057  271.60  30.82  34.71  32.77
9           21  1982    10  0.057  273.10  27.66  36.79  32.23
10          21  1982    11  0.063  275.28  26.28  34.48  30.38
11          21  1982    12  0.074  277.68  25.86  36.39  31.12
12          21  1982    13  0.085  279.65  22.76  40.53  31.65
13          21  1982    14  0.098  281.32  18.26  46.96  32.61
14          21  1982    15  0.107  282.33  11.26  59.73

#### Вибірка: Ряд VHI для області за вказаний рік

In [116]:
def get_vhi_for_region_year(folder_path, region_num, year):
    combined_data = data_frame(folder_path)  # Отримуємо всі дані одним викликом
    
    # Фільтруємо тільки потрібний регіон і рік
    vhi_data = combined_data[(combined_data['region_num'] == region_num) & (combined_data['Year'].astype(int) == year)]
    
    if vhi_data.empty:
        print(f"Немає даних VHI для області {region_num} за {year} рік.")
        return None
    
    return vhi_data[['Year', 'Week', 'VHI']]

In [118]:
result = get_vhi_for_region_year('vhi_data', 21, 2025)
print(result.head(10))

      Year  Week    VHI
2186  2025     1  41.80
2187  2025     2  44.23
2188  2025     3  45.71
2189  2025     4  46.91
2190  2025     5  47.34
2191  2025     6  47.38


#### Вибірка: Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [147]:
def calculate_extremes_and_statistics(vhi_data, region_num):
    # Якщо дані є
    if vhi_data is not None and not vhi_data.empty:
        # Отримуємо назву області
        region_name = regions_true_id.get(region_num, "Невідома область")

        # Розрахунок мінімуму та максимуму
        vhi_min = vhi_data['VHI'].min()
        vhi_max = vhi_data['VHI'].max()

        # Розрахунок середнього та медіани
        vhi_mean = round(vhi_data['VHI'].mean(), 2)
        vhi_median = round(vhi_data['VHI'].median(), 2)

        # Виведення результатів
        print(f"Область: {region_name}")
        print(f"Мінімум VHI: {vhi_min}")
        print(f"Максимум VHI: {vhi_max}")
        print(f"Середнє VHI: {vhi_mean}")
        print(f"Медіана VHI: {vhi_median}")
    else:
        print("Немає даних для розрахунків.")


In [148]:
vhi_data = get_vhi_for_region_year('vhi_data', 1, 2020)
calculate_extremes_and_statistics(vhi_data, 1)


Область: Вінницька
Мінімум VHI: 34.48
Максимум VHI: 64.12
Середнє VHI: 45.91
Медіана VHI: 44.23


#### Вибірка: Ряд VHI за вказаний діапазон років для вказаних областей

In [168]:
def get_vhi_for_regions_year_range(folder_path, region_nums, start_year, end_year):
    combined_data = data_frame(folder_path)  # Отримуємо всі дані

    # Фільтруємо за діапазоном років та областями
    vhi_data = combined_data[(combined_data['region_num'].isin(region_nums)) & 
                              (combined_data['Year'].astype(int) >= start_year) & 
                              (combined_data['Year'].astype(int) <= end_year)]

    return vhi_data[['Year', 'region_num', 'VHI']] if not vhi_data.empty else None

In [169]:
vhi_data = get_vhi_for_regions_year_range('vhi_data', [1, 2, 3], 2001, 2002)
if vhi_data is not None:
    print(vhi_data)
else:
    print("Немає даних для вказаного періоду та областей.")


       Year  region_num    VHI
33838  2001           1  36.50
33839  2001           1  39.49
33840  2001           1  43.40
33841  2001           1  42.84
33842  2001           1  41.60
...     ...         ...    ...
49281  2002           3  63.10
49282  2002           3  63.30
49283  2002           3  64.08
49284  2002           3  66.34
49285  2002           3  64.59

[312 rows x 3 columns]


#### Вибірка: Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI

In [170]:
import pandas as pd

def detect_extreme_drought_years(df, vhi_threshold=15, affected_percentage=20):
    total_regions = df['region_num'].nunique()
    min_affected_regions = int((affected_percentage / 100) * total_regions)
    extreme_drought_years = {}

    # Фільтрація даних по умовам посухи
    extreme_conditions = df[df['VHI'] < vhi_threshold]

    # Групування даних за роками та підрахунок кількості областей з екстремальними посухами
    drought_counts_by_year = extreme_conditions.groupby('Year')['region_num'].nunique()

    for year, affected_count in drought_counts_by_year.items():
        if affected_count >= min_affected_regions:
            drought_areas = extreme_conditions[extreme_conditions['Year'] == year][['region_num', 'VHI']]
            # Використовуємо .drop_duplicates для уникнення дублювання областей
            drought_areas = drought_areas.drop_duplicates(subset='region_num')
            drought_areas['region_name'] = drought_areas['region_num'].map(regions_true_id)  # Переклад номерів областей у назви
            extreme_drought_years[year] = drought_areas[['region_name', 'VHI']]

    return extreme_drought_years

def format_extreme_drought_output(drought_data):
    output = []
    for year, drought_areas in drought_data.items():
        regions_info = [f"{row['region_name']} (VHI={row['VHI']:.2f})" for _, row in drought_areas.iterrows()]
        output.append(f"{year} (Посуха в {len(drought_areas)} областях):\n" + "\n".join(regions_info))
    return "\n\n".join(output)


In [171]:
drought_data = detect_extreme_drought_years(data_frame('vhi_data'), vhi_threshold=15, affected_percentage=20)

if drought_data:
    print(format_extreme_drought_output(drought_data))
else:
    print("Не було років, коли більше 5 областей пережили екстремальні посухи.")

2000 (Посуха в 4 областях):
Київська (VHI=12.51)
Черкаська (VHI=14.64)
Вінницька (VHI=12.26)
Харківська (VHI=14.61)

2007 (Посуха в 5 областях):
Миколаївська (VHI=11.44)
Одеська (VHI=11.88)
Закарпатська (VHI=14.26)
Республіка Крим (VHI=14.98)
Херсонська (VHI=12.41)
